This notebook does a few things:
- Combine NEMSIS v2 and v2 data
- Combine standard and provisional CDC data
- Calculate the fraction of NEMSIS events (per region/month) that were opioid-related (as determined by medications given or ICD codes).
- Standardize date formats, region names, etc.
- Write the results to CSVs.

# Setup

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Data cleaning: Nemsis data
Main goal here is to take counts of Naloxone events or events with opioid-overdose related ICDs, and total NEMSIS event counts (all broken down by region and month) and calculate the fraction of opioid-related events for each region/month.

In [0]:
nemsis_v2_nat_totals_df = pd.read_csv('gdrive/My Drive/nemsis_v2_national_monthly_totals_edited.csv', thousands=',')
nemsis_v2_reg_totals_df = pd.read_csv('gdrive/My Drive/nemsis_v2_regional_monthly_totals_edited.csv', thousands=',')
nemsis_v3_nat_totals_df = pd.read_csv('gdrive/My Drive/nemsis_v3_national_monthly_totals_edited.csv', thousands=',')
nemsis_v3_reg_totals_df = pd.read_csv('gdrive/My Drive/nemsis_v3_regional_monthly_totals_edited.csv', thousands=',')

In [22]:
nemsis_v2_nat_totals_df.head()

,Month,Count of Events
0,1/1/2014,2243815
1,2/1/2014,1987887
2,3/1/2014,2015797
3,4/1/2014,2020800
4,5/1/2014,2313754


In [23]:
nemsis_v2_reg_totals_df

,Month,Census Region,Count of Events
0,1/1/2014,Island Areas,1907
1,1/1/2014,Midwest,401987
2,1/1/2014,Northeast,506985
3,1/1/2014,South,1052501
4,1/1/2014,West,280435
...,...,...,...
175,12/1/2016,Island Areas,2089
176,12/1/2016,Midwest,339919
177,12/1/2016,Northeast,347437
178,12/1/2016,South,1009427


In [24]:
nemsis_v3_nat_totals_df.head()

,Year,Month,Date,Count of Events
0,2017,1,1/1/2017,726630
1,2017,10,10/1/2017,1067091
2,2017,11,11/1/2017,1070661
3,2017,12,12/1/2017,1266258
4,2017,2,2/1/2017,710724


In [25]:
nemsis_v3_reg_totals_df.head()

,Year,Month,Date,Geographic Region of Incident.US Census Regions,Count of Events
0,2017,1,1/1/2017,Midwest,115233
1,2017,1,1/1/2017,Northeast,68291
2,2017,1,1/1/2017,Not Recorded,19508
3,2017,1,1/1/2017,South,279087
4,2017,1,1/1/2017,Territories,649


In [0]:
nemsis_v3_nat_totals_df = nemsis_v3_nat_totals_df.drop(['Year', 'Month'], axis=1)
nemsis_v3_reg_totals_df = nemsis_v3_reg_totals_df.drop(['Year', 'Month'], axis=1)

In [0]:
nemsis_v3_nat_totals_df = nemsis_v3_nat_totals_df.rename(columns={"Date": "Month"})
nemsis_v3_reg_totals_df = nemsis_v3_reg_totals_df.rename(columns={"Date": "Month", "Geographic Region of Incident.US Census Regions": "Region"})

In [0]:
nemsis_v3_reg_totals_df = nemsis_v3_reg_totals_df.rename(columns={"Census Region": "Region"})

In [0]:
nemsis_v2_reg_totals_df = nemsis_v2_reg_totals_df.rename(columns={"Census Region": "Region"})

In [0]:
nemsis_nat_totals_df = nemsis_v2_nat_totals_df.append(nemsis_v3_nat_totals_df)
nemsis_reg_totals_df = nemsis_v2_reg_totals_df.append(nemsis_v3_reg_totals_df)

In [38]:
nemsis_nat_totals_df.head()

,Month,Total Events
0,2014-01-01,2243815
1,2014-02-01,1987887
2,2014-03-01,2015797
3,2014-04-01,2020800
4,2014-05-01,2313754


In [39]:
nemsis_reg_totals_df.head()

,Month,Region,Total Events
0,2014-01-01,Island Areas,1907
1,2014-01-01,Midwest,401987
2,2014-01-01,Northeast,506985
3,2014-01-01,South,1052501
4,2014-01-01,West,280435


In [0]:
nemsis_nat_totals_df[["Month"]] = nemsis_nat_totals_df[["Month"]].apply(pd.to_datetime)
nemsis_reg_totals_df[["Month"]] = nemsis_reg_totals_df[["Month"]].apply(pd.to_datetime)

In [0]:
nemsis_nat_totals_df = nemsis_nat_totals_df.rename(columns={"Count of Events": "Total Events"})
nemsis_reg_totals_df = nemsis_reg_totals_df.rename(columns={"Count of Events": "Total Events"})

In [0]:
nemsis_nat_totals_df.to_csv('nemsis_v2v3_national_monthly_totals.csv')
nemsis_reg_totals_df.to_csv('nemsis_v2v3_regional_monthly_totals.csv')

In [0]:
nemsis_nat_med_df = pd.read_csv('gdrive/My Drive/nemsis_v2v3_monthly_medication.csv', thousands=',')
nemsis_reg_med_df = pd.read_csv('gdrive/My Drive/nemsis_v2v3_monthly_reg_medication.csv', thousands=',')
nemsis_nat_icd_df = pd.read_csv('gdrive/My Drive/nemsis_v2v3_monthly_icd.csv', thousands=',')
nemsis_reg_icd_df = pd.read_csv('gdrive/My Drive/nemsis_v2v3_monthly_reg_icd.csv', thousands=',')

In [41]:
nemsis_nat_med_df.head()

,Month,Count of Events
0,2014-01-01,9940
1,2014-02-01,9196
2,2014-03-01,9870
3,2014-04-01,10060
4,2014-05-01,11590


In [42]:
nemsis_reg_med_df.head()

,Month,Region,Count of Events
0,2014-01-01,Midwest,1982
1,2014-01-01,Northeast,1425
2,2014-01-01,South,4673
3,2014-01-01,Territories,1
4,2014-01-01,West,1859


In [43]:
nemsis_nat_icd_df.head()

,Month,Count of Events
0,2014-01-01,32802
1,2014-02-01,32020
2,2014-03-01,35085
3,2014-04-01,34500
4,2014-05-01,39711


In [44]:
nemsis_reg_icd_df.head()

,Month,Region,Count of Events
0,2014-01-01,Midwest,7593
1,2014-01-01,Northeast,6973
2,2014-01-01,South,10868
3,2014-01-01,Territories,9
4,2014-01-01,West,7359


In [0]:
nemsis_nat_med_df[["Month"]] = nemsis_nat_med_df[["Month"]].apply(pd.to_datetime)
nemsis_reg_med_df[["Month"]] = nemsis_reg_med_df[["Month"]].apply(pd.to_datetime)
nemsis_nat_icd_df[["Month"]] = nemsis_nat_icd_df[["Month"]].apply(pd.to_datetime)
nemsis_reg_icd_df[["Month"]] = nemsis_reg_icd_df[["Month"]].apply(pd.to_datetime)

In [0]:
nemsis_nat_med_df = nemsis_nat_med_df.merge(nemsis_nat_totals_df, how='outer', on='Month')
nemsis_reg_med_df = nemsis_reg_med_df.merge(nemsis_reg_totals_df, how='outer', on=['Month', 'Region'])
nemsis_nat_icd_df = nemsis_nat_icd_df.merge(nemsis_nat_totals_df, how='outer', on='Month')
nemsis_reg_icd_df = nemsis_reg_icd_df.merge(nemsis_reg_totals_df, how='outer', on=['Month', 'Region'])

In [0]:
for df in [nemsis_nat_med_df, nemsis_reg_med_df, nemsis_nat_icd_df, nemsis_reg_icd_df]:
  df[["Count of Events"]] = df[["Count of Events"]].apply(pd.to_numeric)
  df[["Total Events"]] = df[["Total Events"]].apply(pd.to_numeric)
  df['Ratio'] = df['Count of Events'] / df['Total Events']
  # Just for ease of plotting
  df['Ratio_1m'] = df['Count of Events'] / df['Total Events'] * 1000000
  df['Ratio_100k'] = df['Count of Events'] / df['Total Events'] * 100000

In [48]:
nemsis_nat_icd_df.tail()

,Month,Count of Events,Total Events,Ratio,Ratio_1m,Ratio_100k
66,2019-07-01,7760,2609045,0.002974,2974.268363,297.426836
67,2019-08-01,7742,2284163,0.003389,3389.425361,338.942536
68,2019-09-01,7061,2063875,0.003421,3421.234329,342.123433
69,2019-10-01,6066,1864195,0.003254,3253.951437,325.395144
70,2019-11-01,6,278207,0.000022,21.566675,2.156668


In [0]:
# Drop counts from last month of data, which are incomplete
nemsis_nat_med_df = nemsis_nat_med_df[nemsis_nat_med_df['Month'] < '2019-11-01']
nemsis_nat_icd_df = nemsis_nat_icd_df[nemsis_nat_icd_df['Month'] < '2019-11-01']
nemsis_reg_med_df = nemsis_reg_med_df[nemsis_reg_med_df['Month'] < '2019-11-01']
nemsis_reg_icd_df = nemsis_reg_icd_df[nemsis_reg_icd_df['Month'] < '2019-11-01']

In [0]:
nemsis_reg_med_df = nemsis_reg_med_df.dropna(subset=['Count of Events'])
nemsis_reg_icd_df = nemsis_reg_icd_df.dropna(subset=['Count of Events'])

In [51]:
nemsis_nat_icd_df.tail()

,Month,Count of Events,Total Events,Ratio,Ratio_1m,Ratio_100k
65,2019-06-01,7695,2260668,0.003404,3403.861160,340.386116
66,2019-07-01,7760,2609045,0.002974,2974.268363,297.426836
67,2019-08-01,7742,2284163,0.003389,3389.425361,338.942536
68,2019-09-01,7061,2063875,0.003421,3421.234329,342.123433
69,2019-10-01,6066,1864195,0.003254,3253.951437,325.395144


In [52]:
nemsis_reg_icd_df.tail()

,Month,Region,Count of Events,Total Events,Ratio,Ratio_1m,Ratio_100k
353,2019-10-01,Midwest,995.0,294142.0,0.003383,3382.719911,338.271991
354,2019-10-01,Northeast,1173.0,320772.0,0.003657,3656.802963,365.680296
355,2019-10-01,Not Recorded,869.0,34429.0,0.025240,25240.349705,2524.034971
356,2019-10-01,South,2526.0,831971.0,0.003036,3036.163520,303.616352
357,2019-10-01,West,503.0,380799.0,0.001321,1320.906830,132.090683


In [0]:
nemsis_nat_med_df.to_csv('nemsis_national_med_with_ratio.csv')
nemsis_reg_med_df.to_csv('nemsis_regional_med_with_ratio.csv')
nemsis_nat_icd_df.to_csv('nemsis_national_icd_with_ratio.csv')
nemsis_reg_icd_df.to_csv('nemsis_regional_icd_with_ratio.csv')

# Data cleaning: CDC data

In [0]:
cdc_df = pd.read_csv('gdrive/My Drive/cdc_opioids_region_clean.csv')

In [54]:
cdc_df.head()

,Census Region,Month,Month Code,Deaths
0,Census Region 1: Northeast,"Jan., 2014",2014/01,894
1,Census Region 1: Northeast,"Feb., 2014",2014/02,793
2,Census Region 1: Northeast,"Mar., 2014",2014/03,872
3,Census Region 1: Northeast,"Apr., 2014",2014/04,751
4,Census Region 1: Northeast,"May, 2014",2014/05,849


In [0]:
cdc_df['Census Region'] = np.where(cdc_df['Census Region']!='United States', cdc_df['Census Region'].str[17:], cdc_df['Census Region'])

In [0]:
cdc_df[["Month"]] = cdc_df[["Month Code"]].apply(pd.to_datetime)

In [0]:
cdc_df = cdc_df.drop('Month Code', axis=1)

In [58]:
cdc_df.head()

,Census Region,Month,Deaths
0,Northeast,2014-01-01,894
1,Northeast,2014-02-01,793
2,Northeast,2014-03-01,872
3,Northeast,2014-04-01,751
4,Northeast,2014-05-01,849


In [0]:
cdc_df.to_csv('cdc_opioid_region_clean.csv')

# Data cleaning: CDC provisional

In [0]:
cdc_prov_df = pd.read_csv('gdrive/My Drive/cdc-prov-by-region.csv', usecols=['REGION', 'Date', 'Deaths'],
                          parse_dates=['Date'])

In [60]:
cdc_prov_df.head()

,REGION,Date,Deaths
0,United States,2017-01-01,6594
1,United States,2017-02-01,5916
2,United States,2017-03-01,6524
3,United States,2017-04-01,6196
4,United States,2017-05-01,6350


In [0]:
cdc_prov_df = cdc_prov_df.rename(columns={"Date": "Month"})
cdc_prov_df = cdc_prov_df.rename(columns={"REGION": "Census Region"})

In [0]:
# Remove the rows that overlap with our existing non-provisional data
cdc_prov_df = cdc_prov_df[cdc_prov_df['Month'] >= '2018-01-01']

In [63]:
cdc_prov_df.head()

,Census Region,Month,Deaths
12,United States,2018-01-01,6020
13,United States,2018-02-01,5542
14,United States,2018-03-01,6152
15,United States,2018-04-01,5882
16,United States,2018-05-01,6106


In [0]:
cdc_df.to_csv('cdc_prov_region_clean.csv')

In [0]:
cdc_df = cdc_df.append(cdc_prov_df)

In [0]:
cdc_df.to_csv('cdc_combined_region_clean.csv')